In [6]:
import arviz as az
import numpy as np
import pymc3 as pm


"""Data Generation"""
x = np.linspace(0, 10, 20) # creates 20 evenly spaced numbers between 0 and 10
noise = np.random.normal(0, 1, 20) # creates 20 random numbers from a normal distribution with mean 0 and standard deviation 1
y = 1.5 * x + 5 + noise # creates 20 y values, slope = 1.5, intercept = 5 + noise

"""Remove Data to Simulate Missing Data"""
x[10] = np.nan # Remove one x value
y[5] = np.nan # Remove one y value

In [8]:
with pm.Model() as ex1:
    alpha = pm.Normal("alpha", sigma=100)
    beta = pm.Normal("beta", sigma=100)
    tau = pm.Gamma("tau", alpha=0.001, beta=0.001)

    """Impute Missing Data"""
    # PyMC will pick up on missing data and "fill it in" according to the following prior
    # The non-missing values will be the same as the observed data
    # Only the missing values will be imputed
    # Final result is a vector of 20 values (for a given sampling iteration)
    X_imputed = pm.Normal("X_imputed", 5, 5, observed=x)

    mu = alpha + beta * X_imputed

    pm.Normal("likelihood", mu=mu, tau=tau, observed=y)

    trace = pm.sample(draws=3000)

/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/pymc3/model.py:1755: ImputationWarning: Data in X_imputed contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/pymc3/model.py:1755: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
M

/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 8 seconds.


In [10]:
az.summary(trace)

Got error No model on context stack. trying to find log_likelihood in translation.
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,3.945,0.599,2.829,5.079,0.008,0.006,6076.0,6262.0,1.0
beta,1.605,0.099,1.423,1.797,0.001,0.001,6068.0,5934.0,1.0
X_imputed_missing[0],5.937,0.829,4.357,7.452,0.009,0.007,8660.0,6678.0,1.0
likelihood_missing[0],8.144,1.366,5.498,10.626,0.015,0.011,8111.0,6985.0,1.0
tau,0.680,0.238,0.286,1.144,0.003,0.003,5187.0,4006.0,1.0
